In [1]:
import numpy as np
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from itertools import product

In [2]:
import numpy as np
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from itertools import product

# a function to get data from any machine learning models and hyperparameters
def get_data(model, hyperparameters, inputs):

    # get every combination of hyperparameters without loops
    keys = list(hyperparameters)
    hyperparameters_list = list(product(*(hyperparameters[key] for key in keys)))
    # change it into dictionary
    hyperparameters_list = [dict(zip(keys, values)) for values in hyperparameters_list]
    
    y_new = np.zeros((len(inputs), len(hyperparameters_list)))
    for i, hyperparameter in enumerate(hyperparameters_list):
        reg = model(**hyperparameter)
        reg.fit(X[:, np.newaxis],y)
        y_new[:, i] = reg.predict(inputs[:, np.newaxis])

    y_new = pd.DataFrame(y_new.reshape(len(inputs), len(hyperparameters_list)))
    # split every columns to lists
    tmp = []
    for i in range(len(hyperparameters_list)):
        tmp.append(y_new[i])
    # reshape
    y_new = np.array(tmp).reshape(len(inputs)*len(hyperparameters_list),1)

    return y_new


def get_title(hyperparameters):
    # get title for any number of hyperparameters
    keys = list(hyperparameters)
    title = ''
    for i in range(len(keys)):
        title += keys[i] + ' = ' + str(hyperparameters[keys[i]])[1:-1] + ', '
    return title[:-2]


def vis(X, y, model, param_grid, input):
    
    # make a copy of param_grid and get the keys
    h_temp = param_grid.copy()
    keys = list(param_grid)

    # get the sliders for each hyperparameter
    sliders = {}
    for h in range(len(param_grid)):
        param_name = keys[h]
        sliders[param_name] = widgets.SelectionSlider(options=param_grid[keys[h]], value=param_grid[keys[h]][0], description=param_name)

    # function to make the plot
    @interact(**sliders)
    def plot_series(**kwargs):
        for h in range(len(param_grid)):
            param_name = keys[h]
            slider_value = kwargs[param_name]
            h_temp[param_name] = [slider_value]
        y_new = get_data(model, h_temp, input)
        plt.scatter(X,y,label='training data')
        plt.plot(np.linspace(0, 1, 100),true_fun(np.linspace(0, 1, 100)),label='true function')
        plt.plot(input,y_new.reshape(len(input),),'r',label='prediction')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(get_title(h_temp))
        plt.legend()

    return

In [3]:
#initialization
np.random.seed(10)
n_samples = 30
def true_fun(X):
    return np.cos(1.5 * np.pi * X)
X = np.random.rand(n_samples)
y = true_fun(X) + np.random.randn(n_samples) * 0.1

In [4]:
# Try the example from the meterial
from sklearn.ensemble import RandomForestRegressor
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
    'n_estimators': [1, 10,20,30,40],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

vis(X, y, RandomForestRegressor, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='n_estimators', options=(1, 10, 20, 30, 40), value=1), Selec…

In [5]:
# Try SVR
from sklearn.svm import SVR
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
    'gamma': [1e5, 1e3, 1e1, 1e-1,1e-3],
    'C': [1e-3,1e-1,1e1,1e3]
}

vis(X, y, SVR, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='gamma', options=(100000.0, 1000.0, 10.0, 0.1, 0.001), value…

In [6]:
# Try only one hyperparameter
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
    'gamma': [1e5, 1e3, 1e1, 1e-1,1e-3]
}

vis(X, y, SVR, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='gamma', options=(100000.0, 1000.0, 10.0, 0.1, 0.001), value…

In [7]:
# Try 3 hyperparameters with a string hyperparameter
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
    'bootstrap': [True, False],
    'n_estimators': [1, 10,20,30,40],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
}

vis(X, y, RandomForestRegressor, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='bootstrap', options=(True, False), value=True), SelectionSl…

In [8]:
# Try one more hyperparameter
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
    'bootstrap': [True, False],
    'n_estimators': [1, 10,20,30,40],
    'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'max_features': ['auto', 'sqrt']
}

vis(X, y, RandomForestRegressor, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='bootstrap', options=(True, False), value=True), SelectionSl…

In [9]:
# Try one more different model
import xgboost as xgb
X_new = np.linspace(0, 1, 1000)
hyperparameters = {
        'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
        'colsample_bylevel':[0.5,0.6,0.7,0.8,0.9,1.0],
        'learning_rate':[0.01,0.05,0.1,0.2,0.3]
}

vis(X, y, xgb.XGBRegressor, hyperparameters, X_new)

interactive(children=(SelectionSlider(description='max_depth', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=…